In [26]:
import sqlalchemy
import pandas as pd

# Connecting to AWS Database

In [17]:
host="sqlalchemytutorial.cqfrmkvci5hl.eu-central-1.rds.amazonaws.com"
port=3306
dbname="TUTORIAL"
user="admin"
password="SQLAlchemytutorial"

In [24]:
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'
            .format(user, password,host, dbname)).connect()

In [ ]:
from sqlalchemy_utils.functions import database_exists, create_database

# Preparing csv data for Relational Database

In [83]:
data=pd.read_csv("London_property_data.csv")

In [84]:
data.head()

,id,MonthlyPrice,Currency,bedrooms,bathrooms,latitude,longitude,download_time,City,Conutry
0,87559256,1000.0,GBP,1.0,1.0,51.424110,-0.054950,2020-10-04 12:09:00.062859,London,UK
1,72990621,1000.0,GBP,1.0,1.0,51.534100,-0.201950,2020-10-04 12:09:00.062859,London,UK
2,70875120,1057.0,GBP,3.0,1.0,51.525940,-0.162630,2020-10-04 12:09:00.062859,London,UK
3,83795935,1000.0,GBP,NaN,1.0,51.456526,-0.200109,2020-10-04 12:09:00.062859,London,UK
4,72973739,1040.0,GBP,NaN,NaN,51.549730,-0.183310,2020-10-04 12:09:00.062859,London,UK


In [85]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1035 entries, 0 to 1034
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             1035 non-null   int64  
 1   MonthlyPrice   1034 non-null   float64
 2   Currency       1035 non-null   object 
 3   bedrooms       486 non-null    float64
 4   bathrooms      788 non-null    float64
 5   latitude       1035 non-null   float64
 6   longitude      1035 non-null   float64
 7   download_time  1035 non-null   object 
 8   City           1035 non-null   object 
 9   Conutry        1035 non-null   object 
dtypes: float64(5), int64(1), object(4)
memory usage: 81.0+ KB


In [86]:
data.id = data.id.astype(int)

In [87]:
data.columns

Index(['id', 'MonthlyPrice', 'Currency', 'bedrooms', 'bathrooms', 'latitude',
       'longitude', 'download_time', 'City', 'Conutry'],
      dtype='object')

In [88]:
property_features=['id','bedrooms', 'bathrooms']
location_features=['id','latitude','longitude','City', 'Conutry']
price_features=['id', 'MonthlyPrice','download_time']

In [89]:
table_properties=data[property_features]
table_locations=data[location_features]
table_prices=data[price_features]

In [90]:
table_properties.head()

,id,bedrooms,bathrooms
0,87559256,1.0,1.0
1,72990621,1.0,1.0
2,70875120,3.0,1.0
3,83795935,NaN,1.0
4,72973739,NaN,NaN


In [91]:
table_locations.head()

,id,latitude,longitude,City,Conutry
0,87559256,51.424110,-0.054950,London,UK
1,72990621,51.534100,-0.201950,London,UK
2,70875120,51.525940,-0.162630,London,UK
3,83795935,51.456526,-0.200109,London,UK
4,72973739,51.549730,-0.183310,London,UK


In [92]:
table_prices.head()

,id,MonthlyPrice,download_time
0,87559256,1000.0,2020-10-04 12:09:00.062859
1,72990621,1000.0,2020-10-04 12:09:00.062859
2,70875120,1057.0,2020-10-04 12:09:00.062859
3,83795935,1000.0,2020-10-04 12:09:00.062859
4,72973739,1040.0,2020-10-04 12:09:00.062859


# Writing data to database

In [81]:
table_property.to_sql(con=database_connection, name="PROPERTIES", if_exists='replace',chunksize=100)
table_location.to_sql(con=database_connection, name="LOCATIONS", if_exists='replace',chunksize=100)
table_prices.to_sql(con=database_connection, name="PRICES", if_exists='replace',chunksize=100)

# Querying data with SQL

In [99]:
sql_query='''
SELECT * FROM PRICES a
JOIN LOCATIONS b
ON a.id=b.id
JOIN PROPERTIES c
ON a.id=c.id
'''

In [101]:
property_data=pd.read_sql(sql_query,con=database_connection)

In [102]:
property_data.head()

,index,id,MonthlyPrice,download_time,index,id,latitude,longitude,City,Conutry,index,id,bedrooms,bathrooms
0,0,87559256,1000.0,2020-10-04 12:09:00.062859,0,87559256,51.424110,-0.054950,London,UK,0,87559256,1.0,1.0
1,1,72990621,1000.0,2020-10-04 12:09:00.062859,1,72990621,51.534100,-0.201950,London,UK,1,72990621,1.0,1.0
2,2,70875120,1057.0,2020-10-04 12:09:00.062859,2,70875120,51.525940,-0.162630,London,UK,2,70875120,3.0,1.0
3,3,83795935,1000.0,2020-10-04 12:09:00.062859,3,83795935,51.456526,-0.200109,London,UK,3,83795935,NaN,1.0
4,4,72973739,1040.0,2020-10-04 12:09:00.062859,4,72973739,51.549730,-0.183310,London,UK,4,72973739,NaN,NaN


In [103]:
property_data.shape

(1035, 14)

# Creating new schema

In [22]:
dbname_2='TUTORIAL_ADVANCED'
engin_uri = 'mysql+mysqlconnector://{0}:{1}@{2}/{3}'.format(user, password,host, dbname_2)

if not database_exists(engin_uri):
    create_database(engin_uri)